In [1]:
import keras
import numpy as np
import pandas as pd 
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature
import mlflow

In [2]:
data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";", )

data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
train,test=train_test_split(data,test_size=0.25,random_state=42)   

In [4]:
train[['quality']].values.ravel()

array([6, 6, 5, ..., 6, 6, 8])

In [5]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

## test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

## splitting this train data into train and validation

train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)

In [6]:
def train_model(params, train_x, train_y, valid_x, valid_y):
    # Model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    
    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(1)
    ])

    # Compile model with the specified hyperparameters from params
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["learning_rate"],
            momentum=params["momentum"]
        ),
        loss='mean_squared_error',
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    # Train model
    model.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=3, batch_size=64)

    # Evaluate model
    eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
    eval_rmse = eval_result[1]

    # Log parameters and metrics
    mlflow.log_params(params)
    mlflow.log_metric("eval_rmse", eval_rmse)
    mlflow.tensorflow.log_model(model, "ANN model", signature=signature)

    return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [12]:
def objective(params):
    # Start a new MLflow run for each trial
    with mlflow.start_run(nested=True):
        # Call the train_model function with the parameters
        result = train_model(
            params=params,
            train_x=train_x,
            train_y=train_y,
            valid_x=valid_x,
            valid_y=valid_y
        )
    return result


In [13]:
space = {
    'learning_rate': hp.uniform('learning_rate', 0.00001, 0.1),
    "momentum": hp.uniform("momentum", 0.0, 1.0)
}

mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 22s 501ms/step - loss: 32.7631 - root_mean_squared_error: 5.7239
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - loss: 25.9201 - root_mean_squared_error: 5.0458
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - loss: 22.4383 - root_mean_squared_error: 4.6751
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - loss: 19.8907 - root_mean_squared_error: 4.3813
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 18.0294 - root_mean_squared_error: 4.1557
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 16.5441 - root_mean_squared_error: 3.8054   
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 15.3329 - root_mean_squared_error: 3.8054   
23/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.9134 - root_mean_squared_error: 2.6501
22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.1243 - root_mean_squared_error: 2.6139
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.5302 - root_mean_squared_error: 2.5467
24/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.715

In [15]:
## Inferencing

from mlflow.models import validate_serving_input

model_uri = 'runs:/0dcef790417a4ed7a80bfbc5216f3719/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

c:\Users\Ganesh Mahadev\OneDrive\Desktop\MLFlow\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[6.1419277],
       [6.4991417],
       [6.2586555],
       ...,
       [6.1264   ],
       [6.397646 ],
       [5.7207804]], dtype=float32)

In [16]:
# Load model as a PyFuncModel.
model_uri = 'runs:/0dcef790417a4ed7a80bfbc5216f3719/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


array([[6.1419277],
       [6.4991417],
       [6.2586555],
       ...,
       [6.1264   ],
       [6.397646 ],
       [5.7207804]], dtype=float32)

In [17]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1730591658453, current_stage='None', description=None, last_updated_timestamp=1730591658453, name='wine-quality', run_id='0dcef790417a4ed7a80bfbc5216f3719', run_link=None, source='file:///c:/Users/Ganesh%20Mahadev/OneDrive/Desktop/MLFlow/dlproject/mlruns/524334706822896100/0dcef790417a4ed7a80bfbc5216f3719/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>